In [21]:
# Stability analysis of n-forest system
# requires rule for jacobian for fixed-point analysis
using Random
using PyPlot
using ChaosTools
using DrWatson
using UnPack
# @quickactivate :CSSim
include(joinpath(srcdir(), "n_forest.jl"))

antonovsky_sym_jacob

In [23]:
?TangentDynamicalSystem

search: TangentDynamicalSystem



```
TangentDynamicalSystem <: DynamicalSystem
TangentDynamicalSystem(ds::CoreDynamicalSystem; kwargs...)
```

A dynamical system that bundles the evolution of `ds` (which must be an [`CoreDynamicalSystem`](@ref)) and `k` deviation vectors that are evolved according to the *dynamics in the tangent space* (also called linearized dynamics or the tangent dynamics).

The state of `ds` **must** be an `AbstractVector` for `TangentDynamicalSystem`.

`TangentDynamicalSystem` follows the [`DynamicalSystem`](@ref) interface with the following adjustments:

  * `reinit!` takes an additional keyword `Q0` (with same default as below)
  * The additional functions [`current_deviations`](@ref) and [`set_deviations!`](@ref) are provided for the deviation vectors.

## Keyword arguments

  * `k` or `Q0`: `Q0` represents the initial deviation vectors (each column = 1 vector). If `k::Int` is given, a matrix `Q0` is created with the first `k` columns of the identity matrix. Otherwise `Q0` can be given directly as a matrix. It must hold that `size(Q, 1) == dimension(ds)`. You can use [`orthonormal`](@ref) for random orthonormal vectors. By default `k = dimension(ds)` is used.
  * `u0 = current_state(ds)`: Starting state.
  * `J` and `J0`: See section "Jacobian" below.

## Description

Let $u$ be the state of `ds`, and $y$ a deviation (or perturbation) vector. These two are evolved in parallel according to

$$
\begin{array}{rcl}
\frac{d\vec{x}}{dt} &=& f(\vec{x}) \\
\frac{dY}{dt} &=& J_f(\vec{x}) \cdot Y
\end{array}
\quad \mathrm{or}\quad
\begin{array}{rcl}
\vec{x}_{n+1} &=& f(\vec{x}_n) \\
Y_{n+1} &=& J_f(\vec{x}_n) \cdot Y_n.
\end{array}
$$

for continuous or discrete time respectively. Here $f$ is the [`dynamic_rule`](@ref)`(ds)` and $J_f$ is the Jacobian of $f$.

## Jacobian

The keyword `J` provides the Jacobian function. It must be a Julia function in the same form as `f`, the [`dynamic_rule`](@ref). Specifically, `J(u, p, n) -> M::SMatrix` for the out-of-place version or `J(M, u, p, n)` for the in-place version acting in-place on `M`. in both cases `M` is a matrix whose columns are the deviation vectors.

By default `J = nothing`.  In this case `J` is constructed automatically using the module [`ForwardDiff`](https://github.com/JuliaDiff/ForwardDiff.jl), hence its limitations also apply here. Even though `ForwardDiff` is very fast, depending on your exact system you might gain significant speed-up by providing a hand-coded Jacobian and so it is recommended. Additionally, automatic and in-place Jacobians cannot be time dependent.

The keyword `J0` allows you to pass an initialized Jacobian matrix `J0`. This is useful for large in-place systems where only a few components of the Jacobian change during the time evolution. `J0` can be a sparse or any other matrix type. If not given, a matrix of zeros is used. `J0` is ignored for out of place systems.


In [32]:
?fixedpoints

search: fixedpoints



```
fixedpoints(ds::CoreDynamicalSystem, box, J = nothing; kwargs...) → fp, eigs, stable
```

Return all fixed points `fp` of the given out-of-place `ds` (either `DeterministicIteratedMap` or `CoupledODEs`) that exist within the state space subset `box` for parameter configuration `p`. Fixed points are returned as a [`Dataset`](@ref). For convenience, a vector of the Jacobian eigenvalues of each fixed point, and whether the fixed points are stable or not, are also returned.

`box` is an appropriate `IntervalBox` from IntervalRootFinding.jl. E.g. for a 3D system it would be something like

```julia
v, z = -5..5, -2..2   # 1D intervals, can use `interval(-5, 5)` instead
box = v × v × z       # `\times = ×`, or use `IntervalBox(v, v, z)` instead
```

`J` is the Jacobian of the dynamic rule of `ds`. It is like in [`TangentDynamicalSystem`](@ref), however in this case automatic Jacobian estimation does not work, hence a hand-coded version must be given.

Internally IntervalRootFinding.jl is used and as a result we are guaranteed to find all fixed points that exist in `box`, regardless of stability. Since IntervalRootFinding.jl returns an interval containing a unique fixed point, we return the midpoint of the interval as the actual fixed point. Naturally, limitations inherent to IntervalRootFinding.jl apply here.

The output of `fixedpoints` can be used in the [BifurcationKit.jl](https://github.com/rveltz/BifurcationKit.jl) as a start of a continuation process. See also [`periodicorbits`](@ref).

## Keyword arguments

  * `method = IntervalRootFinding.Krawczyk` configures the root finding method, see the docs of IntervalRootFinding.jl for all posibilities.
  * `tol = 1e-15` is the root-finding tolerance.
  * `warn = true` throw a warning if no fixed points are found.


In [34]:
# Inspect jacobian for two forest system
# ... note that authors assume a₂ = 0
n = 1
J_n = n_forest_sym_jacob(n)
display(size(J_n))
display(J_n)

(2, 2)

2×2 Matrix{Num}:
 a₁*α[1] - c - f - a*((y[1] - b)^2)  ρ - 2a*(y[1] - b)*x[1]
               f                                a₂*α[1] - h

In [31]:
# Example indices that will need to be modified 
J = Matrix(undef, 2*n, 2*n)
state_ix = 1
for i in 1:2:2*n
        println("α_$(state_ix), y_$(state_ix), x_$(state_ix)" *
            "-->\n$(i),$(i); $(i),$(i+1);\n$(i+1),$(i); $(i+1),$(i+1);\n------")
        state_ix += 1
end 

α_1, y_1, x_1-->
1,1; 1,2;
2,1; 2,2;
------
α_2, y_2, x_2-->
3,3; 3,4;
4,3; 4,4;
------
α_3, y_3, x_3-->
5,5; 5,6;
6,5; 6,6;
------
